# My Notes 

## Research papers of interest
These papers are stored at the Arxiv website, they are mostly have not been through peer reviews because they are so new. On the other hand, they are follow up and performed by others researchers and interested users on the internet, if there are problems these will be report almost instantly on the social network.

[diffedit](https://arxiv.org/abs/2210.11427)  
[Progressive Distillation for Fast Sampling of Diffusion Models](https://arxiv.org/abs/2202.00512)  
[On Distillation of Guided Diffusion Models](https://arxiv.org/abs/2210.03142)  
[Imagic: Text-Based Real Image Editing with Diffusion Models](https://arxiv.org/abs/2210.09276)  



## Eistein Summation

A technique for convenient and simplify the writing of syntax for many operation in linear algebra

- Matrix multiplication
- Element-wise matrix operation
- Permutation of matrix
- Dot product of matrix
- Outer product of matrix
- Summation of matrix
- Batch multiplication of matrix (permute input to match function calls ordering)

It's also speedup some of the above operation especially operation that can be combined into single call

The eisum is a build-in feature of most Machine learning frameworks, e.g. Pytorch, Tensorflow, Flux...

How does it work?

Here is the matrix multiplication  
$$ M_{ij} = \sum{A_{ik}B_{kj}} = A_{ik}B_{kj}$$

In [1]:
import numpy as np

In [2]:
A = np.random.rand(3,5)
B = np.random.rand(5,2)
M = np.empty((3,2))

A.shape,B.shape,M.shape

((3, 5), (5, 2), (3, 2))

In [3]:
A,B,M

(array([[0.25751708, 0.35363583, 0.44094689, 0.15923051, 0.94069891],
        [0.6454825 , 0.82196553, 0.90130844, 0.86911307, 0.74164791],
        [0.71329793, 0.09704208, 0.87791006, 0.55405125, 0.81099   ]]),
 array([[0.69920459, 0.6770418 ],
        [0.49278349, 0.87058154],
        [0.18258558, 0.18972417],
        [0.42383985, 0.68421629],
        [0.62110089, 0.25180904]]),
 array([[4.65926309e-310, 0.00000000e+000],
        [6.90464571e-310, 6.90464444e-310],
        [6.90464693e-310, 6.90464444e-310]]))

In [4]:
np.set_printoptions(precision=2, linewidth=140)

### Code:  
Coding as loop of the matrix multiplication above   
where the row is i and column is j, and k is the inner dimension of both matrix that must be equal and this index will be summed and disappeared by the operation of matrix rule

In [5]:
for i in range(3):
    for j in range(2):
        total = 0
        for k in range(5):
            total += A[i,k] * B[k,j]
        M[i,j] = total

print(f'the matrix is: {M}')

the matrix is: [[1.09 0.91]
 [1.85 2.11]
 [1.45 1.32]]


Using the Eisum method

In [6]:

# the i and j are free index
# the k is sum index since it will be summed away after the operation
M1 = np.einsum('ik,kj->ij',A,B)

print(f'the matix is : {M1}')

the matix is : [[1.09 0.91]
 [1.85 2.11]
 [1.45 1.32]]


### Dot product
Vector or matrix multiplicaton $u \cdot v$  that result of scalar value  

### Outer product 
vector multiply by vector that result in a matrix $u \otimes v$  

Example 2
- Using the free index in the output  
- No summation index  

In [7]:
# example 2
D = np.random.rand(5)
E = np.random.rand(3)
out = np.einsum('i,j->ij',D,E)
print(f'the matrix is: \n{out}')

the matrix is: 
[[0.17 0.32 0.12]
 [0.22 0.41 0.15]
 [0.15 0.29 0.11]
 [0.02 0.04 0.01]
 [0.06 0.12 0.04]]


In [8]:
# loop version
for i in range(5):
    for j in range(3):
        total = 0
        total += D[i] * E[j]
        out[i,j] = total
        
print(f'the matrix is: \n{out}')

the matrix is: 
[[0.17 0.32 0.12]
 [0.22 0.41 0.15]
 [0.15 0.29 0.11]
 [0.02 0.04 0.01]
 [0.06 0.12 0.04]]


## The Eisum Rules

The free indices: 
- The index that specify the output

The Summation index  
- All other indices that appear in the input argument but not show up in the output

The General rules:
1. Same index in the a different input argument indicate that these indices will be multiplied and the product are outputed
``` python
    M = np.einsum('ik,kj->ij',A,B)
```
2. Omitting index indicate the index will be summed together
``` python
    X = np.ones(3)
    Y = np.einsum('i->',X)
```
3. The unsummed indices may return in any order
``` python
    D = np.ones((5,4,3))
    E = np.einsum('ijk->kji',D)
```

Operation that benefit from Einsum
1. Permutation of Tensors
2. Summation
3. Column sum
4. Row sum
5. Matrix-Vector multiplication
6. Matrix-Matrix multiplication
7. Dot Product the first row with first row of a matrix
8. Dot product with matrix (multiplication and add)
9. Element-wise multiplication (Hadamard Product) (multiplication no add)
10. Outer Product
11. Batch matrix multiplicaton e.g. a = 3,2,6 and b = 3,6,3
    - want to multiply the matrix of 2x6 with 6x3 matrix
    - these matrix must follow the multiplication rule
    - the first number  is the batch size they must match, but not count as index 
    - the torch.bmm function will do the same thing
12. Matrix diagonal  
    - return the only the diagonal value of the matrix
13. Matrix Trace
    - summing the value of the diagonal of a matrix

In [9]:
# !pip install torch

In [10]:
import torch
import numpy as np

torch.set_printoptions(precision=2, linewidth=140)

X = torch.rand((2,3))
X

tensor([[0.70, 0.80, 0.61],
        [0.45, 0.17, 0.65]])

1. Transpose
Switching the index of a matrix

In [11]:
#| label: mat-transpose
#| code-fold: True

c_ntp = np.transpose(X)
c_tp = torch.transpose(X,0,1)
cein = torch.einsum('ij->ji',X)
print(f'numpy: {c_ntp} \npytorch: {c_tp}\n \neinsum: {cein}')

numpy: tensor([[0.70, 0.45],
        [0.80, 0.17],
        [0.61, 0.65]]) 
pytorch: tensor([[0.70, 0.45],
        [0.80, 0.17],
        [0.61, 0.65]])
 
einsum: tensor([[0.70, 0.45],
        [0.80, 0.17],
        [0.61, 0.65]])


2. Matrix summation  
Summing all value in the matrix that result in a scalar value

In [12]:
#| label: matrix-sum

#2. Summation
cma = torch.sum(X)
cein = torch.einsum('ij->',X)

print(f'regular: {cma} \neinsum: {cein}')

regular: 3.374739646911621 
einsum: 3.374739646911621


In [13]:
X

tensor([[0.70, 0.80, 0.61],
        [0.45, 0.17, 0.65]])

Row sum (Top down)

Add all values from each row together along the column

In [14]:
#| label: row-summation

#4 row summation
# sum by columns
rows = torch.sum(X,dim=0)
cein = torch.einsum('ij->j',X)

print(f'regular: {rows} \neinsum: {cein}')

regular: tensor([1.14, 0.97, 1.26]) 
einsum: tensor([1.14, 0.97, 1.26])


Column sum (Left to right)

Add all value from each column together along the row

In [15]:
#| label: column-summation

#3 Column summation
# sum by rows
c_col = torch.sum(X,dim=1)
cein = torch.einsum('ij->i',X)

print(f'regular: {c_col} \neinsum: {cein}')

regular: tensor([2.11, 1.27]) 
einsum: tensor([2.11, 1.27])


5 matrix-vector multiplication  
This a non equal dimension multiplication which in Python use broadcasting to padded (duplicate) the vector to have equal size with the matrix before do multiplication

In [16]:
#| label: matrix-vector

#5 matrix-vector multiplication
L = torch.rand((1,3))
M = torch.rand((3,))

cmm = torch.matmul(L,M)
cein = torch.einsum('ij,j->i',L,M)
print(f'regular: {cmm} \neinsum: {cein}')


regular: tensor([0.45]) 
einsum: tensor([0.45])


6 matrix-matrix multiplication  
This standard matrix to matrix multiplication which result in another matrix

In [17]:
#| label: matrix-matrix

#6 matrix-matrix multiplication
# torch.einsum('ij,kj->ik',M,M)

a = torch.ones((3,2))
b = torch.ones((2,3))
cmm = torch.matmul(a,b)
cein = torch.einsum('ij,jl->il',a,b)
print(f'regular: {cmm} \neinsum: {cein}')


regular: tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]]) 
einsum: tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]])


In [18]:
N = torch.rand((3,3))
M = torch.rand((2,3))
torch.einsum('ij,kj->ik',N,M)

tensor([[0.98, 1.00],
        [1.00, 0.73],
        [1.56, 1.48]])

Dot product

This a matrix multiplication that result in a scalar value. It's usually called multiply add.  
since after multiply the row to the column then the sum operation is carry out result as a scalar

In [19]:
#| label: dot-product

# dot product of a matrix (multiply + add)
#torch.einsum('ij,ij->',N,N)

# c = torch.tensor([2,3])
# d = torch.tensor([2,1])
c = torch.rand((3))
d = torch.rand((3))

c_dot = torch.dot(c,d)
cein = torch.einsum('i,i->',c,d)

print(f'c: {c}, c: {c.shape}')
print(f'c_dot: {c_dot}')
print(f'regular: {c_dot} \n  einsum: {cein}')

c: tensor([0.43, 0.15, 0.87]), c: torch.Size([3])
c_dot: 0.47965747117996216
regular: 0.47965747117996216 
  einsum: 0.47965747117996216


In [20]:
# dot product of only the first row of a matrix with first row of a matrix
torch.einsum('i,i->',N[0],N[0])

tensor(0.80)

Hadamard Product 
Element wise multiplication (multiply only)  
This is a normal matrix multiplication which different from multiply add or dot product

In [21]:
#| label: Hadamard-product

# element wise multiplication (multiply only)
# torch.einsum('ij,ij->ij',N,N)

c = torch.randn((3,2))
d = torch.randn((3,2))
cmm = c * d
cein = torch.einsum('ij,ij->ij',c,d)
print(f'regular: {cmm} \n  einsum: {cein}')

regular: tensor([[ 0.10,  0.67],
        [-0.81,  0.05],
        [-0.75, -0.17]]) 
  einsum: tensor([[ 0.10,  0.67],
        [-0.81,  0.05],
        [-0.75, -0.17]])


Outer Product

Multiply vector of different size to get a matrix as output
In eisum must use different letter to represent size different


In [22]:
#| label: outer-product

# outer product (inner product)
x = torch.rand(3)
y = torch.rand(5)
print(f'x: {x}, x: {x.shape}')
print(f'y: {y}, y: {y.shape}')

c_outer = torch.outer(x,y)
cein = torch.einsum('i,j->ij',x,y)
print(f'regular: {c_outer} \n  einsum: {cein}')

x: tensor([0.43, 0.57, 0.64]), x: torch.Size([3])
y: tensor([0.28, 0.29, 0.36, 0.29, 0.80]), y: torch.Size([5])
regular: tensor([[0.12, 0.13, 0.16, 0.13, 0.35],
        [0.16, 0.16, 0.21, 0.16, 0.45],
        [0.18, 0.19, 0.23, 0.18, 0.51]]) 
  einsum: tensor([[0.12, 0.13, 0.16, 0.13, 0.35],
        [0.16, 0.16, 0.21, 0.16, 0.45],
        [0.18, 0.19, 0.23, 0.18, 0.51]])


Batch matrix multiplication  
Multiply matrix by the set of n, where n is batch size  

want to multiply the matrix of 2x6 with 6x3 matrix  
the first number is the batch size must match but not count as index so i is ignore  
the mxn * nxp must match with n

In [23]:
#| label: batch-matrix

# batch matrix multiplicaton
# want to multiply the matrix of 2x6 with 6x3 matrix
# the first number is the batch size must match but not count as index so i is ignore
# the mxn * nxp must match with n
R = torch.rand(3,2,6)
S = torch.rand(3,6,3)
cmn = np.matmul(R,S)
cmm = torch.matmul(R,S)

cein = torch.einsum('ijk,ikl->ijl',R,S)

print(f'regular: {cmm}\n numpy: {cmn} \n  einsum: {cein}')

regular: tensor([[[1.50, 2.31, 2.54],
         [0.58, 0.66, 0.85]],

        [[1.78, 2.65, 1.25],
         [0.62, 1.06, 0.59]],

        [[0.84, 1.63, 1.22],
         [0.75, 1.63, 1.19]]])
 numpy: tensor([[[1.50, 2.31, 2.54],
         [0.58, 0.66, 0.85]],

        [[1.78, 2.65, 1.25],
         [0.62, 1.06, 0.59]],

        [[0.84, 1.63, 1.22],
         [0.75, 1.63, 1.19]]]) 
  einsum: tensor([[[1.50, 2.31, 2.54],
         [0.58, 0.66, 0.85]],

        [[1.78, 2.65, 1.25],
         [0.62, 1.06, 0.59]],

        [[0.84, 1.63, 1.22],
         [0.75, 1.63, 1.19]]])


Diagonal Matrix
return the vector of value along the diagonal of a matrix

In [24]:
#| label: diag-mat

# Diagonal matrix return only the diagonal value of a matrix

T = torch.rand(3,3)

cein = torch.einsum('ii->i',T)
print(f'T: {T} \nT shape: {T.shape}')
c_diag = torch.diag(T)

print(f'regular: {c_diag} \n  einsum: {cein}')


T: tensor([[0.53, 0.41, 0.31],
        [0.07, 0.23, 0.83],
        [0.98, 0.96, 0.52]]) 
T shape: torch.Size([3, 3])
regular: tensor([0.53, 0.23, 0.52]) 
  einsum: tensor([0.53, 0.23, 0.52])


In [25]:
# torch.einsum('ii->i',T)

Trace

Take the sum of all values along the diagonal axix of a matrix

In [26]:
#| label: trace

# matrix trace

c_trace = torch.trace(T)
cein = torch.einsum('ii->',T)
print(f'T: {T}')
print(f'regular: {c_trace} \n  einsum: {cein}')

T: tensor([[0.53, 0.41, 0.31],
        [0.07, 0.23, 0.83],
        [0.98, 0.96, 0.52]])
regular: 1.275879144668579 
  einsum: 1.275879144668579


Tensor Contraction  
Shrinking the dimension of the tensor  
must provide the dimension to be ignored


In [27]:
#| label: contraction

o = torch.rand((3,4,2))
p = torch.rand((4,3,6))
print(f'value: {o.shape} value2: {p.shape}')

c_tdot = torch.tensordot(o,p,dims=([1,0],[0,1]))
cein = torch.einsum('ijk,jil->kl',o,p)
print(f'regular: {c_tdot} \n  einsum: {cein}')

value: torch.Size([3, 4, 2]) value2: torch.Size([4, 3, 6])
regular: tensor([[3.03, 3.33, 3.06, 2.75, 4.21, 2.40],
        [3.15, 2.79, 2.38, 2.85, 3.71, 2.13]]) 
  einsum: tensor([[3.03, 3.33, 3.06, 2.75, 4.21, 2.40],
        [3.15, 2.79, 2.38, 2.85, 3.71, 2.13]])


Bilinear transformation

In [28]:
a = torch.rand(2,3)
b = torch.rand(5,3,7)
c = torch.rand(2,7)

torch.einsum('ik,jkl,il->ij',[a,b,c])

tensor([[0.89, 1.05, 1.08, 1.24, 1.05],
        [3.68, 3.52, 2.98, 3.65, 3.15]])